In [1]:
import asyncio
import requests
from concurrent.futures import ThreadPoolExecutor
from timeit import default_timer
import json
import requests

In [2]:
access_token = "BQAJSl-kC8iA9AppLyMfw722iQP26BGEviwN0zPRokiglornvzylw8vgnif-E16mrGX7ZJdGq_5zZuQZUBY_nxIGHbFGnutkqM-ICoG4GlVlCygQO5cJ-aXMTPNUbogQdssDvKn5JC1HgV8OWSU4DB-JAqII0XhOf9ZBedlcJxvSYDlDLg"
authorization_header = {"Authorization": "Bearer {}".format(access_token)}

In [3]:
SPOTIFY_API_BASE_URL = "https://api.spotify.com"
API_VERSION = "v1"
SPOTIFY_API_URL = "{}/{}".format(SPOTIFY_API_BASE_URL, API_VERSION)
user_profile_api_endpoint = "{}/me".format(SPOTIFY_API_URL)

user_tracks_endpoint = "{}/tracks".format(user_profile_api_endpoint)

In [4]:
def fetch(session, query):
    base_url = user_tracks_endpoint
    url = base_url + query
    authorization_header = {"Authorization": "Bearer {}".format(access_token)}
    with session.get(url, headers=authorization_header) as response:
        data = response.json()
        if response.status_code != 200:
            print("FAILURE::{0}".format(url))
            
        elapsed = default_timer() - START_TIME
        time_completed_at = "{:5.2f}s".format(elapsed)
        print("{0:<30} {1:>20}".format(query, time_completed_at))
        
        return data

async def get_data_asynchronous(queries):   
    print("{0:<30} {1:>20}".format("File", "Completed at"))

    # Note: max_workers is set to 10 simply for this example,
    # you'll have to tweak with this number for your own projects
    # as you see fit
    with ThreadPoolExecutor(max_workers=10) as executor:
        with requests.Session() as session:
            # Set any session parameters here before calling `fetch`

            # Initialize the event loop        
            loop = asyncio.get_event_loop()
                            
            # Set the START_TIME for the `fetch` function
            START_TIME = default_timer()

            # Use list comprehension to create a list of
            # tasks to complete. The executor will run the `fetch`
            # function for each csv in the csvs_to_fetch list
            tasks = [
                loop.run_in_executor(
                    executor,
                    fetch,
                    *(session, query) # Allows us to pass in multiple arguments to `fetch`
                )
                for query in queries
            ]
            
            # Initializes the tasks to run and awaits their results
            responses = []
            for response in await asyncio.gather(*tasks):
                responses.append(response)
            return responses


In [5]:
def get_num_tracks():
    response = requests.get(user_tracks_endpoint, headers=authorization_header)
    if response.status_code != 200:
        raise Exception("Got response: {}".format(response.status_code))
    return response.json()['total']

In [6]:
queries = []
num_items = get_num_tracks()
limit = 50
for i in range(int(num_items / 50) + 1):
    queries.append("?offset={}&limit={}".format(i * limit , limit))

In [7]:
START_TIME = default_timer()

loop = asyncio.get_event_loop()
future = asyncio.ensure_future(get_data_asynchronous(queries))

In [8]:
results = await future
user_tracks = sum([result['items'] for result in results], [])

File                                   Completed at
?offset=200&limit=50                          0.25s
?offset=150&limit=50                          0.26s
?offset=50&limit=50                           0.27s
?offset=450&limit=50                          0.29s
?offset=100&limit=50                          0.30s
?offset=350&limit=50                          0.31s
?offset=0&limit=50                            0.32s
?offset=500&limit=50                          0.42s
?offset=550&limit=50                          0.44s
?offset=650&limit=50                          0.45s
?offset=600&limit=50                          0.47s
?offset=800&limit=50                          0.48s
?offset=750&limit=50                          0.50s
?offset=700&limit=50                          0.50s
?offset=850&limit=50                          0.58s
?offset=900&limit=50                          0.60s
?offset=950&limit=50                          0.62s
?offset=1050&limit=50                         0.66s
?offset=1100

In [9]:
len(user_tracks)

3720